In [1]:
%load_ext autoreload
%autoreload 2

from openai import OpenAI
import os
import time
import sys
import numpy as np
import re

from gen_utils import insert_phrase, math_reward
import time

sys.path.append('/proj/dmfexp/statllm/users/mikhail/sunday-ai/deepscaler/deepscaler/rewards/math_utils')
from utils import grade_answer_mathd, grade_answer_sympy, extract_answer

from vllm import LLM, SamplingParams

from transformers import AutoTokenizer

# model_name = "meta-llama/Llama-3.3-70B-Instruct"
# model_name = "meta-llama/Llama-3.1-8B"
# model_name = "Qwen/Qwen2-1.5B"
# model_name = 'Qwen/Qwen2.5-Math-7B'
# model_name = 'Qwen/Qwen2.5-1.5B'
# model_name = 'microsoft/phi-4'
model_name = 'ibm-granite/granite-3.1-8b-instruct'

/proj/dmfexp/statllm/users/mikhail/vllm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-06 21:17:34,499	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
iters = 5
max_try = 8
max_tokens = 4000
n_sample = 16
chat = True

sampling_params = SamplingParams(max_tokens = max_tokens,
                             n = n_sample,
                             temperature = 0.7)

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [3]:
llm = LLM(model=model_name, enable_prefix_caching = True, gpu_memory_utilization = 0.95)

INFO 03-06 21:17:41 config.py:510] This model supports multiple tasks: {'score', 'generate', 'reward', 'embed', 'classify'}. Defaulting to 'generate'.
WARNING 03-06 21:17:41 arg_utils.py:1103] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 03-06 21:17:41 config.py:1458] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 03-06 21:17:41 llm_engine.py:234] Initializing an LLM engine (v0.6.6.post1) with config: model='ibm-granite/granite-3.1-8b-instruct', speculative_config=None, tokenizer='ibm-granite/granite-3.1-8b-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:04<00:13,  4.61s/it]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:09<00:09,  4.68s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:10<00:03,  3.19s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:15<00:00,  3.71s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:15<00:00,  3.82s/it]



INFO 03-06 21:18:05 model_runner.py:1099] Loading model weights took 15.2511 GB
INFO 03-06 21:18:05 worker.py:241] Memory profiling takes 0.58 seconds
INFO 03-06 21:18:05 worker.py:241] the current vLLM instance can use total_gpu_memory (79.21GiB) x gpu_memory_utilization (0.95) = 75.25GiB
INFO 03-06 21:18:05 worker.py:241] model weights take 15.25GiB; non_torch_memory takes 0.17GiB; PyTorch activation peak memory takes 0.49GiB; the rest of the memory reserved for KV Cache is 59.33GiB.
INFO 03-06 21:18:05 gpu_executor.py:76] # GPU blocks: 24301, # CPU blocks: 1638
INFO 03-06 21:18:05 gpu_executor.py:80] Maximum concurrency for 131072 tokens per request: 2.97x
INFO 03-06 21:18:08 model_runner.py:1415] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_util

Capturing CUDA graph shapes: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:09<00:00,  3.50it/s]

INFO 03-06 21:18:18 model_runner.py:1535] Graph capturing finished in 10 secs, took 0.39 GiB
INFO 03-06 21:18:18 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 13.21 seconds


In [4]:
import json

# data_path = '/proj/dmfexp/statllm/users/mikhail/sunday-ai/deepscaler/deepscaler/data/test/math.json'
# data_path = '/proj/dmfexp/statllm/users/mikhail/sunday-ai/deepscaler/deepscaler/data/test/aime.json'
data_path = '/proj/dmfexp/statllm/users/mikhail/sunday-ai/LIMO/train/data/limo.json'

data_name = data_path.split('/')[-1][:-5]

with open(data_path) as f:
    data = json.load(f)

In [11]:
# sys.path.append('/proj/dmfexp/statllm/users/mikhail/sunday-ai/deepscaler/deepscaler/rewards/math_utils')
# from utils import grade_answer_mathd, extract_answer # generated then reference
SYS_PROMPT = """Let's think step by step and output the final answer within \\boxed{}. """

In [12]:
delimiter = '\n\n'
special_phrases = ['Let\'s try another method to solve the problem:', 'Now, let\'s think again:', 'Wait', 'Let\'s doublecheck the work so far.', 'Alternatively', 
                   'Let\'s look at it from a different perspective:']

In [15]:
idx_range = [0,1,2,3,4,5]

for idx in idx_range:

    print('\n' + 20*'==' + f' Datapoint {idx} ' + 20*'==' + '\n')
    
    d = data[idx]

    if data_name == 'limo':
        d['problem'] = d['instruction']
        d['answer'] = extract_answer(d['output'])
    
    # x = {'problem': 'How many times does 0 occur between 1 and 123?', 'answer': '22'}
    print(d['problem'])
    print(d['answer'])

    solutions = []
    cur_answer = ''
    
    if chat:
        prompt_d = tokenizer.apply_chat_template([{"role": "user", 'content': SYS_PROMPT + d['problem']}], tokenize=False, add_generation_prompt=True)
    else:
        prompt_d = SYS_PROMPT + d['problem']
                
    for i in range(iters):
        print('\n' + 10*'++' + f' Iteration {i} ' + 10*'++' + '\n')
        
        found_answer = False
        count_try = 0
        while not found_answer:
            
            t_s = time.time()
            completions_bon = llm.generate(prompt_d + cur_answer, sampling_params, use_tqdm=False)
            print('Generation took', time.time() - t_s)
    
            responses = [out.text for out in completions_bon[0].outputs]
    
            scores = []
            for r in responses:
                if r is None:
                    scores.append(0.)
                else:
                    scores.append(math_reward(cur_answer + r, d['answer']))
                            
    
            if max(scores) == 1:
                found_answer = True
            else:
                count_try += 1
                if count_try == max_try:
                    break
    
        if not found_answer:
            print(f'Faied at iter {i}')
            break
            
        idx_sol = scores.index(1)
        if i < iters - 1:
            cut_response, delimiter_count = insert_phrase(responses[idx_sol], delimiter, special_phrases, max_position = 10)
            cur_answer = cur_answer + cut_response
            cur_answer = re.sub(r'\\boxed\{([^}]+)\}', r'\1', cur_answer)
            print(f'Delimiter count {delimiter_count}')
        else:
            cur_answer = cur_answer + responses[idx_sol]
        print(f'Mean reward {np.mean(scores)}\n')
        solutions.append(cur_answer)
        print(cur_answer)


======================================== Datapoint 0 ========================================

Find the last three digits of the product of the positive roots of $\sqrt{1995}x^{\log_{1995}x}=x^2.$
025

++++++++++++++++++++ Iteration 0 ++++++++++++++++++++

Generation took 11.631262302398682
Delimiter count 5
Mean reward 0.1875
To find the last three digits of the product of the positive roots of the equation $\sqrt{1995}x^{\log_{1995}x}=x^2$, we start by simplifying the given equation.

1. **Simplify the Equation:**
   - The equation $\sqrt{1995}x^{\log_{1995}x}=x^2$ can be rewritten using properties of logarithms and exponents. Specifically, the term $x^{\log_{1995}x}$ can be expressed as $1995^{\log_{1995}x \cdot \log_{1995}x}$ due to the change of base formula for logarithms. This simplifies the equation to:
     \[
     \sqrt{1995} \cdot 1995^{\log_{1995}x \cdot \log_{1995}x} = x^2
     \]
   - Since $1995^{\log_{1995}x \cdot \log_{1995}x} = x^{\log_{1995}x}$, we can further simpl